### Glossary  <br>

`Year` -- Year at time of season finale <br>
`Age` -- Player age at time of finale <br>
`Pos` -- Player position <br>
`GP` -- Number of Games Played in season <br>
`G` -- Goals <br>
`A` -- Assists <br>
`PTS` -- Points <br>
`PS` -- Point Shares; an estimate of the number of points contributed by a player. <br>
`PP` -- Power Play Goals <br>
`PP.1` -- Power Play Assists <br>
`S` -- Shots on Goal <br>
`S%` -- Shooting Percentage (total goals / total shots on goal) <br>
`TOI` -- Total Time on Ice (in minutes) <br>
`ATOI` -- Average Time on Ice per game <br>

In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from scipy.stats import ttest_ind
import glob

We are going to add a 'Year' column to each .csv file (each season's data) one-by-one before compiling them all into one large dataframe. <br>


In [131]:
# Reading csv file
nhl2014= pd.read_csv("C:/Users/joshm/Documents/BrainStation/Capstone Project/hockey-reference.com data/nhl2014.csv", index_col=1)

# Inserting a new 'Year' column into df
nhl2014.insert(loc=1, column='Year', value=2014)

# saving our modified .csv file
nhl2014.to_csv("C:/Users/joshm/Documents/BrainStation/Capstone Project/hockey-reference.com data/nhl2014.csv")

In [132]:
nhl2014.info()

<class 'pandas.core.frame.DataFrame'>
Index: 886 entries, 1 to 886
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  886 non-null    object 
 1   Year    886 non-null    int64  
 2   Age     886 non-null    int64  
 3   Tm      886 non-null    object 
 4   Pos     886 non-null    object 
 5   GP      886 non-null    int64  
 6   G       886 non-null    int64  
 7   A       886 non-null    int64  
 8   PTS     886 non-null    int64  
 9   +/-     886 non-null    int64  
 10  PIM     886 non-null    int64  
 11  PS      886 non-null    float64
 12  EV      886 non-null    int64  
 13  PP      886 non-null    int64  
 14  SH      886 non-null    int64  
 15  GW      886 non-null    int64  
 16  EV.1    886 non-null    int64  
 17  PP.1    886 non-null    int64  
 18  SH.1    886 non-null    int64  
 19  S       886 non-null    int64  
 20  S%      862 non-null    float64
 21  TOI     886 non-null    int64  
 22  ATOI   

In [133]:
# Sort by any column
sorted_nhl2018 = nhl2018.sort_values(by='G', ascending=False)
sorted_nhl2018.head(30)

,Rk,Year,Age,Tm,Pos,GP,G,A,PTS,+/-,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,-9999
Player,,,,,,,,,,,,,,,,,,,,,
Alex Ovechkin,11,2018,32,WSH,LW,82,49,38,87,3,...,355,13.8,1652,20:09,21.0,139,3.0,5.0,37.5,ovechal01
Patrik Laine,34,2018,19,WPG,RW,82,44,26,70,8,...,241,18.3,1351,16:29,43.0,74,11.0,32.0,25.6,lainepa01
William Karlsson,23,2018,25,VEG,C,82,43,35,78,49,...,184,23.4,1535,18:43,45.0,48,402.0,435.0,48.0,karlswi01
Evgeni Malkin,4,2018,31,PIT,C,78,42,56,98,16,...,239,17.6,1481,18:59,32.0,48,460.0,592.0,43.7,malkiev01
Eric Staal,28,2018,33,MIN,C,82,42,34,76,8,...,241,17.4,1461,17:49,49.0,42,571.0,544.0,51.2,staaler01
Connor McDavid,1,2018,21,EDM,C,82,41,67,108,20,...,274,15.0,1767,21:33,46.0,28,376.0,533.0,41.4,mcdavco01
Anders Lee,62,2018,27,NYI,LW,82,40,22,62,-25,...,208,19.2,1416,17:16,38.0,102,11.0,5.0,68.8,leean01
Tyler Seguin,24,2018,26,DAL,C,82,40,38,78,12,...,335,11.9,1715,20:55,44.0,74,790.0,650.0,54.9,seguity01
Nikita Kucherov,3,2018,24,TBL,RW,80,39,61,100,15,...,279,14.0,1586,19:49,15.0,31,3.0,2.0,60.0,kucheni01


In [121]:
# applying a filter (in this case gp >= 30) and then sorting.
filtered_nhl23 = nhl23[nhl23['GP'] >= 30]
filtered_nhl23.sort_values(by='Age', ascending=True).head(30)

,Rk,Age,Tm,Pos,GP,G,A,PTS,+/-,PIM,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,-9999
Player,,,,,,,,,,,,,,,,,,,,,
Connor Bedard,102,18,CHI,C,45,17,23,40,-26,14,...,135,12.6,865,19:13,18,32,145,223,39.4,bedarco01
Zach Benson,327,18,BUF,LW,47,6,12,18,-2,22,...,71,8.5,697,14:50,22,19,4,8,33.3,bensoza01
Adam Fantilli,210,19,CBJ,C,49,12,15,27,-21,16,...,115,10.4,770,15:43,25,63,171,221,43.6,fantiad01
Juraj Slafkovsky,185,19,MTL,LW,58,12,18,30,-13,35,...,94,12.8,1010,17:25,51,97,7,10,41.2,slafkju01
Logan Cooley,208,19,ARI,C,57,8,19,27,-11,16,...,94,8.5,898,15:45,23,39,181,261,41.0,coolelo01
Simon Nemec,377,19,NJD,D,37,2,14,16,-3,16,...,49,4.1,744,20:06,60,13,0,0,NaN,nemecsi01
Matthew Poitras,398,19,BOS,C,33,5,10,15,4,6,...,40,12.5,442,13:24,15,17,111,143,43.7,poitrma01
Kevin Korchinski,489,19,CHI,D,53,3,7,10,-23,12,...,63,4.8,1040,19:38,57,14,0,0,NaN,korchke01
Leo Carlsson,262,19,ANA,C,39,8,14,22,-10,12,...,78,10.3,713,18:16,14,25,122,237,34.0,carlsle01


In [252]:
# Combining multiple .csv files into one dataframe

# Get a list of all CSV files in the directory
csv_files = glob.glob("C:/Users/joshm/Documents/BrainStation/Capstone Project/hockey-reference.com data/*.csv")

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over each CSV file
for csv_file in csv_files:
    # Read the CSV file into a DataFrame
    combined_df = pd.read_csv(csv_file, index_col=2)
    # Append the DataFrame to the list
    dfs.append(combined_df)

# Concatenate all DataFrames in the list into one DataFrame
nhl_df = pd.concat(dfs, ignore_index=False)

nhl_df

,Player,Rk,Age,Tm,Pos,GP,G,A,PTS,+/-,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,-9999
Year,,,,,,,,,,,,,,,,,,,,,
2014,Sidney Crosby,1,26.0,PIT,C,80,36,68,104,18,...,259,13.9,1758,21:58,28.0,47,991.0,896.0,52.5,crosbsi01
2014,Ryan Getzlaf,2,28.0,ANA,C,77,31,56,87,28,...,204,15.2,1639,21:17,87.0,110,692.0,719.0,49.0,getzlry01
2014,Claude Giroux,3,26.0,PHI,C,82,28,58,86,7,...,223,12.6,1676,20:26,46.0,90,931.0,829.0,52.9,giroucl01
2014,Tyler Seguin,4,22.0,DAL,C,80,37,47,84,16,...,294,12.6,1548,19:21,24.0,47,280.0,395.0,41.5,seguity01
2014,Corey Perry,5,28.0,ANA,RW,81,43,39,82,32,...,280,15.4,1578,19:29,28.0,81,9.0,16.0,36.0,perryco01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,Chris Wagner,865,32.0,COL,C,4,0,0,0,0,...,2,0.0,28,6:54,0.0,11,7.0,3.0,70.0,wagnech01
2024,Samuel Walker,866,24.0,MIN,C,4,0,0,0,-3,...,3,0.0,37,9:22,1.0,2,0.0,1.0,0.0,walkesa01
2024,Colin White,867,27.0,TOT,C,13,0,0,0,-1,...,10,0.0,131,10:02,5.0,13,16.0,13.0,55.2,whiteco02


In [253]:
filtered_nhl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6861 entries, Sidney Crosby to Oskar Steen
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    6861 non-null   int64  
 1   Age     6861 non-null   float64
 2   Pos     6861 non-null   object 
 3   GP      6861 non-null   int64  
 4   G       6861 non-null   int64  
 5   A       6861 non-null   int64  
 6   PTS     6861 non-null   int64  
 7   PS      6861 non-null   float64
 8   PP      6861 non-null   int64  
 9   PP.1    6861 non-null   int64  
 10  S       6861 non-null   int64  
 11  S%      6861 non-null   float64
 12  TOI     6861 non-null   int64  
 13  ATOI    6861 non-null   object 
dtypes: float64(3), int64(9), object(2)
memory usage: 804.0+ KB


In [254]:
# Dropping unwanted columns
nhl_df.drop(columns=['Rk','Tm','+/-','PIM','EV','SH','GW','EV.1','SH.1','BLK','HIT','FOW','FOL','FO%','-9999'], inplace=True)
nhl_df

,Player,Age,Pos,GP,G,A,PTS,PS,PP,PP.1,S,S%,TOI,ATOI
Year,,,,,,,,,,,,,,
2014,Sidney Crosby,26.0,C,80,36,68,104,13.5,11,27,259,13.9,1758,21:58
2014,Ryan Getzlaf,28.0,C,77,31,56,87,11.2,5,18,204,15.2,1639,21:17
2014,Claude Giroux,26.0,C,82,28,58,86,10.2,7,30,223,12.6,1676,20:26
2014,Tyler Seguin,22.0,C,80,37,47,84,11.6,11,14,294,12.6,1548,19:21
2014,Corey Perry,28.0,RW,81,43,39,82,12.3,8,10,280,15.4,1578,19:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,Chris Wagner,32.0,C,4,0,0,0,0.0,0,0,2,0.0,28,6:54
2024,Samuel Walker,24.0,C,4,0,0,0,-0.2,0,0,3,0.0,37,9:22
2024,Colin White,27.0,C,13,0,0,0,-0.3,0,0,10,0.0,131,10:02


In [255]:
# Renaming some columns
nhl_df.rename(columns={'PS':'PtSh','PP':'PPG','PP.1':'PPA', 'S':'Shots'}, inplace=True)
nhl_df

,Player,Age,Pos,GP,G,A,PTS,PtSh,PPG,PPA,Shots,S%,TOI,ATOI
Year,,,,,,,,,,,,,,
2014,Sidney Crosby,26.0,C,80,36,68,104,13.5,11,27,259,13.9,1758,21:58
2014,Ryan Getzlaf,28.0,C,77,31,56,87,11.2,5,18,204,15.2,1639,21:17
2014,Claude Giroux,26.0,C,82,28,58,86,10.2,7,30,223,12.6,1676,20:26
2014,Tyler Seguin,22.0,C,80,37,47,84,11.6,11,14,294,12.6,1548,19:21
2014,Corey Perry,28.0,RW,81,43,39,82,12.3,8,10,280,15.4,1578,19:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,Chris Wagner,32.0,C,4,0,0,0,0.0,0,0,2,0.0,28,6:54
2024,Samuel Walker,24.0,C,4,0,0,0,-0.2,0,0,3,0.0,37,9:22
2024,Colin White,27.0,C,13,0,0,0,-0.3,0,0,10,0.0,131,10:02


In [256]:
new_nhl_df.isnull().sum()

Year     0
Age      0
Pos      0
GP       0
G        0
A        0
PTS      0
PtSh     0
PPG      0
PPA      0
Shots    0
S%       0
TOI      0
ATOI     0
dtype: int64

In [257]:
new_nhl_df = nhl_df[nhl_df['GP'] >= 30]
new_nhl_df.sort_values(by='Age', ascending=True)

,Player,Age,Pos,GP,G,A,PTS,PtSh,PPG,PPA,Shots,S%,TOI,ATOI
Year,,,,,,,,,,,,,,
2014,Nathan MacKinnon,18.0,C,82,24,39,63,7.7,8,9,241,10.0,1422,17:21
2020,Kaapo Kakko,18.0,RW,66,10,13,23,0.5,2,11,109,9.2,942,14:17
2019,Andrei Svechnikov,18.0,RW,82,20,17,37,3.9,0,5,189,10.6,1201,14:39
2024,Connor Bedard,18.0,C,45,17,23,40,3.6,2,9,135,12.6,865,19:13
2017,Patrik Laine,18.0,RW,73,36,28,64,8.9,9,5,204,17.6,1308,17:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,Jaromír Jágr,43.0,RW,79,27,39,66,9.2,5,6,143,18.9,1349,17:05
2014,Teemu Selänne*,43.0,RW,64,9,18,27,2.5,4,4,111,8.1,904,14:08
2021,Zdeno Chára,43.0,D,55,2,8,10,2.2,0,0,67,3.0,1008,18:19


In [264]:
# This is how we can look at the mean of a particular player's stats
new_nhl_df.groupby('Player').mean(numeric_only=True).sort_values(by='PTS', ascending=False).head(30)

,Age,GP,G,A,PTS,PtSh,PPG,PPA,Shots,S%,TOI
Player,,,,,,,,,,,
Connor McDavid,23.0,69.111111,36.000000,68.333333,104.333333,12.233333,8.444444,27.111111,236.111111,15.100000,1501.444444
Nikita Kucherov,24.8,70.300000,31.400000,51.700000,83.100000,9.950000,8.700000,21.900000,215.300000,14.380000,1313.800000
Artemi Panarin,28.0,72.111111,27.666667,54.555556,82.222222,9.400000,7.222222,17.888889,192.000000,14.488889,1418.333333
Sidney Crosby,31.0,70.454545,31.272727,50.272727,81.545455,9.690909,9.090909,17.818182,217.545455,14.454545,1443.181818
Leon Draisaitl,23.5,69.300000,33.400000,48.100000,81.500000,9.390000,13.900000,14.600000,183.500000,16.820000,1419.900000
Patrick Kane,29.5,73.400000,30.200000,51.100000,81.300000,9.070000,7.800000,17.900000,259.800000,11.560000,1532.000000
Mitch Marner,22.5,70.375000,23.875000,54.375000,78.250000,8.212500,5.250000,18.625000,184.000000,12.987500,1395.875000
Nathan MacKinnon,23.0,69.818182,28.909091,48.818182,77.727273,8.900000,8.181818,16.818182,277.181818,10.200000,1413.272727
Auston Matthews,22.5,67.125000,43.875000,33.500000,77.375000,10.375000,11.375000,10.250000,268.250000,16.600000,1325.625000


In [262]:
# This is how we can look at total or means across the years
new_nhl_df.groupby('Year').mean(numeric_only=True).sort_values(by='Year', ascending=False).head(30)

,Age,GP,G,A,PTS,PtSh,PPG,PPA,Shots,S%,TOI
Year,,,,,,,,,,,
2024,27.785235,50.327181,9.124161,15.248322,24.372483,2.711074,1.931208,3.659396,88.449664,9.291443,850.637584
2023,27.395137,67.560790,12.188450,20.545593,32.734043,3.595593,2.568389,4.892097,119.875380,9.215653,1131.565350
2022,27.184132,65.657186,11.829341,19.934132,31.763473,3.515269,2.294910,4.419162,118.577844,9.076497,1104.368263
2021,26.968586,48.045375,8.254799,13.832461,22.087260,2.645201,1.652705,3.193717,83.375218,9.240314,814.963351
2020,26.833333,59.089109,10.264026,17.054455,27.318482,3.177888,2.082508,3.988449,106.579208,8.897855,997.021452
2019,26.534268,66.750779,11.495327,19.303738,30.799065,3.541900,2.247664,4.275701,119.803738,8.636916,1120.397196
2018,26.608764,67.341158,11.306729,19.051643,30.358372,3.560407,2.392801,4.583725,122.123631,8.515493,1127.733959
2017,26.811688,67.433442,10.594156,17.675325,28.269481,3.596429,2.245130,4.279221,115.922078,8.397078,1131.100649
2016,27.009740,67.087662,10.362013,17.310065,27.672078,3.575812,2.288961,4.337662,113.842532,8.253571,1126.662338
